<a href="https://colab.research.google.com/github/z1mmz/ML-Notebooks/blob/master/Dolly_doc_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"
%pip install chromadb
%pip install sentence_transformers
!pip install langchain

  Using cached torch-1.13.1-cp310-cp310-manylinux1_x86_64.whl (887.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.0.1
    Uninstalling torch-2.0.1:
      Successfully uninstalled torch-2.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.6.1 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchtext 0.15.2 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
torchvision 0.15.2+cu118 requires torch==2.0.1, but you have torch 1.13.1 which is incompatible.
  Using cached torch-2.0.1-cp310-cp310-manylinux1_x86_64.whl (619.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.13.1
    Uninstalling torch-1.13.1:
      Successfully uninstalled torch-1.13.1


In [2]:
!pip install unstructured
!pip install pypdf

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!pip install einops

In [5]:

from langchain.llms import HuggingFacePipeline

import torch
from transformers import pipeline

pipe = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
                         trust_remote_code=True, device_map="auto", return_full_text=True)

llm = HuggingFacePipeline(pipeline=pipe)

In [6]:
llm

HuggingFacePipeline(cache=None, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, pipeline=<transformers_modules.databricks.dolly-v2-3b.f6c9be08f16fe4d3a719bee0a4a7c7415b5c65df.instruct_pipeline.InstructionTextGenerationPipeline object at 0x789755630c10>, model_id='gpt2', model_kwargs=None, pipeline_kwargs=None)

In [7]:
!pip install selenium

In [15]:
from langchain.document_loaders import OnlinePDFLoader
from langchain.document_loaders import BSHTMLLoader
from langchain.document_loaders import WebBaseLoader

loaders =[]
documents = []
loaders +=[ OnlinePDFLoader("https://parlinfo.aph.gov.au/parlInfo/download/chamber/hansardr/26709/toc_pdf/House%20of%20Representatives_2023_06_22.pdf;fileType=application%2Fpdf")]


for loader in loaders:
    documents.extend(loader.load())

In [26]:
from langchain.chains import RetrievalQA
from langchain.indexes import VectorstoreIndexCreator
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma

# split the documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=300)
texts = text_splitter.split_documents(documents)
# select which embeddings we want to use
model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

embeddings = hf
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":2})


# create a chain to answer questions
qa = RetrievalQA.from_chain_type(
    llm=llm, chain_type="refine", retriever=retriever,return_source_documents=True, verbose=True)

In [31]:
qa({"query": "Was anyone concered about AI?"})






> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Was anyone concered about AI?',
 'result': '\nyes',
 'source_documents': [Document(page_content='The truth is I have worked in the private sector most of my life; however, I remember a time when I was auditing a public sector organisation and I was also the WA branch president of Professionals Australia, the union that represents engineers. The thing that stood out to me not only when I would do audits but also when spending time with my public sector engineering colleagues was the fundamental values base of those that work in the public sector versus those in the private sector. The thing that I observed was a sense of collegiality, not only wanting to make sure that there is an investment in the skills of the future of the Public Service but also this fundamental desire to contribute to public good.', metadata={'source': '/tmp/tmp1b957t7p/tmp.pdf'}),
  Document(page_content='The truth is I have worked in the private sector most of my life; however, I remember a time when I

In [32]:
qa({"query": "Who was concered about AI?"})




> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Who was concered about AI?',
 'result': '\nThe Australian Defense Science and Technology Group conducted several studies into using artificial intelligence for military purposes in the early twenty-first century.\nThe studies included several possibilities for using artificial intelligence, including:\nScavenging through available artificial intelligence data sources to locate potential targets for attack. Conducting automated pattern-matching analysis of radar imagery and other data to locate targets for other aircraft or weapons to attack. Guiding missiles and bombs using data from artificial intelligence sensors. Despite performing these studies, the Defense Science and Technology Group was unable to make substantial progress toward using artificial intelligence for military purposes.\nDespite performing these studies, the Defense Science and Technology Group was unable to make substantial progress toward using artificial intelligence for military purposes.',
 'source_doc